# Mastering Asynchronous Programming in Python

This guide will help you understand and master async programming in Python, starting from the basics and progressing to advanced concepts.

## Prerequisites
- Basic Python knowledge
- Python 3.7+ installed
- Jupyter Notebook environment

## 1. Core Concepts of Async Programming

### 1.1 Coroutines
Coroutines are the building blocks of async programming in Python. They are defined using the `async def` syntax.

In [1]:
import asyncio

# Basic coroutine
async def simple_coroutine():
    print("Starting")
    await asyncio.sleep(1)  # Simulate some async operation
    print("Finished")

# Run the coroutine
await simple_coroutine()

Starting
Finished


### 1.2 Understanding async/await

- `async`: Declares a coroutine function
- `await`: Pauses execution until an awaitable completes

Key points:
1. You can only use `await` inside `async` functions
2. Coroutines don't run automatically - they must be awaited or scheduled

In [2]:
# Example showing async/await usage
async def fetch_data():
    print("Fetching data...")
    await asyncio.sleep(2)  # Simulate network delay
    return {"data": "example"}

async def process_data():
    data = await fetch_data()  # Wait for fetch_data to complete
    print(f"Processing {data}")

# Run the coroutine
await process_data()

Fetching data...
Processing {'data': 'example'}


### 1.3 Event Loop

The event loop is the core of async programming. It:
1. Manages and schedules coroutines
2. Handles I/O operations
3. Runs tasks concurrently

In [3]:
# Working with the event loop
async def main():
    # Get the current event loop
    loop = asyncio.get_event_loop()
    
    # Schedule multiple coroutines
    task1 = loop.create_task(fetch_data())
    task2 = loop.create_task(fetch_data())
    
    # Wait for both tasks to complete
    await asyncio.gather(task1, task2)

# Run in notebook
await main()

Fetching data...
Fetching data...


## 2. Task Management

Tasks are used to schedule coroutines concurrently. They are crucial for running multiple operations at once.

In [ ]:
async def long_operation(name, seconds):
    print(f"Starting {name}")
    await asyncio.sleep(seconds)
    print(f"Finished {name}")
    return f"{name} completed"

async def task_demo():
    # Create tasks
    task1 = asyncio.create_task(long_operation("Task 1", 2))
    task2 = asyncio.create_task(long_operation("Task 2", 1))
    
    # Wait for both tasks
    results = await asyncio.gather(task1, task2)
    print(f"Results: {results}")

await task_demo()

### 2.1 Common Task Operations

- Creating tasks
- Cancelling tasks
- Handling task results and exceptions

In [ ]:
async def task_operations_demo():
    # Create a task
    task = asyncio.create_task(long_operation("Demo Task", 3))
    
    # Check task status
    print(f"Task done? {task.done()}")
    
    try:
        # Wait for task with timeout
        await asyncio.wait_for(task, timeout=2)
    except asyncio.TimeoutError:
        print("Task took too long, cancelling...")
        task.cancel()
        try:
            await task
        except asyncio.CancelledError:
            print("Task was cancelled")

await task_operations_demo()

## 3. Error Handling in Async Code

Proper error handling is crucial in async programming. Here's how to handle various scenarios:

In [ ]:
async def might_fail(succeed=True):
    await asyncio.sleep(1)
    if not succeed:
        raise ValueError("Operation failed!")
    return "Success!"

async def error_handling_demo():
    try:
        # Basic error handling
        result = await might_fail(succeed=False)
    except ValueError as e:
        print(f"Caught error: {e}")
    
    # Handling multiple concurrent tasks
    tasks = [
        might_fail(succeed=True),
        might_fail(succeed=False)
    ]
    
    # gather() with return_exceptions=True doesn't raise exceptions
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Task {i} failed with: {result}")
        else:
            print(f"Task {i} succeeded with: {result}")

await error_handling_demo()

### 3.1 Best Practices for Error Handling

1. Always use try/except around await statements
2. Handle specific exceptions rather than catching all
3. Consider using return_exceptions=True with gather()
4. Implement proper cleanup in finally blocks

## 4. Async Context Managers

Async context managers are useful for resource management in async code. They use `async with` syntax.

In [ ]:
class AsyncResource:
    async def __aenter__(self):
        print("Acquiring resource")
        await asyncio.sleep(1)  # Simulate resource acquisition
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print("Releasing resource")
        await asyncio.sleep(0.5)  # Simulate cleanup
    
    async def work(self):
        print("Using resource")
        await asyncio.sleep(1)

async def context_manager_demo():
    async with AsyncResource() as resource:
        await resource.work()
    print("Resource has been released")

await context_manager_demo()

### 4.1 Real-world Example: Database Connection

Here's how you might use an async context manager with a database:

In [ ]:
class AsyncDatabase:
    async def __aenter__(self):
        self.conn = await self.connect()
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        await self.conn.close()
    
    async def connect(self):
        await asyncio.sleep(1)  # Simulate connection
        return self
    
    async def query(self, sql):
        await asyncio.sleep(0.5)  # Simulate query
        return ["result1", "result2"]

async def database_demo():
    async with AsyncDatabase() as db:
        results = await db.query("SELECT * FROM table")
        print(f"Query results: {results}")

await database_demo()

## 5. Async Iterators and Generators

Async iterators allow you to iterate over data that's retrieved asynchronously.

In [ ]:
class AsyncDataStream:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.current = start
    
    def __aiter__(self):
        return self
    
    async def __anext__(self):
        if self.current < self.end:
            await asyncio.sleep(0.5)  # Simulate delay
            result = self.current
            self.current += 1
            return result
        raise StopAsyncIteration

async def iterator_demo():
    # Using async for
    async for num in AsyncDataStream(1, 4):
        print(f"Got number: {num}")

await iterator_demo()

### 5.1 Async Generators

Async generators use `async def` with `yield` to create async iterators more easily.

In [ ]:
async def async_range(start, end):
    for i in range(start, end):
        await asyncio.sleep(0.5)  # Simulate delay
        yield i

async def generator_demo():
    # Using async generator
    async for num in async_range(1, 4):
        print(f"Generated: {num}")
    
    # Collecting all results
    results = [num async for num in async_range(5, 8)]
    print(f"Collected: {results}")

await generator_demo()

## 6. Real-world Examples

### 6.1 Web Requests with aiohttp

Here's how to make concurrent HTTP requests using aiohttp:

In [ ]:
import aiohttp

async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def fetch_multiple_urls():
    urls = [
        'http://example.com',
        'http://example.org',
        'http://example.net'
    ]
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        
        for url, html in zip(urls, results):
            print(f"Fetched {len(html)} bytes from {url}")

# Note: This cell requires aiohttp package
# !pip install aiohttp
# await fetch_multiple_urls()

### 6.2 Implementing an Async API Server

Here's a simple async API server using aiohttp:

In [ ]:
from aiohttp import web

async def handle_get(request):
    await asyncio.sleep(0.1)  # Simulate processing
    return web.json_response({"status": "ok"})

async def init_app():
    app = web.Application()
    app.router.add_get("/api/status", handle_get)
    return app

# To run the server:
# web.run_app(init_app())

# Note: This code needs to be run in a regular Python file, not in Jupyter

## 7. Performance Tips and Monitoring

### 7.1 Performance Best Practices

In [ ]:
async def demo_performance_patterns():
    # 1. Batch operations when possible
    async def process_batch(items):
        await asyncio.sleep(1)  # Simulate batch processing
        return [f"processed_{item}" for item in items]
    
    items = list(range(100))
    batch_size = 10
    
    # Process in batches
    for i in range(0, len(items), batch_size):
        batch = items[i:i + batch_size]
        results = await process_batch(batch)
        print(f"Processed batch of {len(results)} items")
    
    # 2. Use asyncio.gather for concurrent operations
    async def parallel_operation(x):
        await asyncio.sleep(0.1)
        return x * 2
    
    tasks = [parallel_operation(x) for x in range(5)]
    results = await asyncio.gather(*tasks)
    print(f"Parallel results: {results}")

await demo_performance_patterns()

### 7.2 Monitoring and Debugging

Here's how to monitor async applications:

In [ ]:
import time
import functools

def async_timer(func):
    @functools.wraps(func)
    async def wrapper(*args, **kwargs):
        start = time.time()
        result = await func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.2f} seconds")
        return result
    return wrapper

@async_timer
async def monitored_operation():
    await asyncio.sleep(1)
    return "done"

# Enable debug mode
asyncio.get_event_loop().set_debug(True)

await monitored_operation()

## 8. Common Pitfalls and Solutions

1. **Mixing sync and async code**
   - Always use async versions of libraries when available
   - Use `asyncio.to_thread()` for CPU-bound tasks
   
2. **Blocking the event loop**
   - Avoid time.sleep() - use asyncio.sleep()
   - Don't perform CPU-intensive operations directly
   
3. **Task management**
   - Always await or cancel tasks
   - Use proper exception handling
   
4. **Resource cleanup**
   - Use async context managers
   - Implement proper cleanup in __aexit__